In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from matplotlib.pyplot import savefig
import math
#plt.use("TkAgg")
#from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import datetime
from tkinter import *
from tkinter import scrolledtext, messagebox, Image, filedialog
from tkinter.ttk import *
import sys
print("import successful")

#####initial definitions######
benfordRatios = [.301,.176,.125,.097,.079,.067,.058,.051,.046]
numberLabels = ['one','two','three','four','five','six','seven','eight','nine']
br = pd.Series(benfordRatios)
nl = pd.Series(numberLabels)

df4 = pd.DataFrame()
df4['1st_Sig_Fig'] = nl.values
df4['Expected Benford Ratios'] = br.values
df4.set_index('1st_Sig_Fig', inplace=True)
ax = df4.plot(linestyle = '-', marker = 'o')
#######Methods############
def get_digit(number):
    n = len(str(number))-1
    return number // 10**n % 10

def compile_ratio_of_first_digits(list1):
    one, two, three, four, five, six, seven, eight, nine = 0,0,0,0,0,0,0,0,0
    for i in range(len(list1)):
        number1 = list1[i]
        number = int(number1)
        firstDigit = get_digit(number)
        if firstDigit == 1:
            one = one + 1
        elif firstDigit == 2:
            two = two + 1
        elif firstDigit == 3:
            three = three + 1
        elif firstDigit == 4:
            four = four + 1
        elif firstDigit == 5:
            five = five + 1
        elif firstDigit == 6:
            six = six + 1
        elif firstDigit == 7:
            seven = seven + 1
        elif firstDigit == 8:
            eight = eight + 1
        elif firstDigit == 9:
            nine = nine + 1
    numberQuantity = one+two+three+four+five+six+seven+eight+nine
    if numberQuantity > 0:
        ratios = [one/numberQuantity,two/numberQuantity,three/numberQuantity,
                  four/numberQuantity,five/numberQuantity,six/numberQuantity,
                  seven/numberQuantity,eight/numberQuantity,nine/numberQuantity]
    else:
        ratios = [0,0,0,0,0,0,0,0,0]
        print("**there were no readable digits in one of the columns")
    return ratios

def compile_quantity_of_first_digits(list1):
    one, two, three, four, five, six, seven, eight, nine = 0,0,0,0,0,0,0,0,0
    for i in range(len(list1)):
        number1 = list1[i]
        number = int(number1)
        firstDigit = get_digit(number)
        if firstDigit == 1:
            one = one + 1
        elif firstDigit == 2:
            two = two + 1
        elif firstDigit == 3:
            three = three + 1
        elif firstDigit == 4:
            four = four + 1
        elif firstDigit == 5:
            five = five + 1
        elif firstDigit == 6:
            six = six + 1
        elif firstDigit == 7:
            seven =  seven + 1
        elif firstDigit == 8:
            eight = eight + 1
        elif firstDigit == 9:
            nine = nine + 1
    numberQuantity = one+two+three+four+five+six+seven+eight+nine
    expectedQuantities = [numberQuantity*.301,numberQuantity*.176,numberQuantity*.125,
                          numberQuantity*.097,numberQuantity*.079,numberQuantity*.067,
                          numberQuantity*.058,numberQuantity*.051,numberQuantity*.046]
    observedQuantities = [one,two,three,four,five,six,seven,eight,nine]
    if one*two*three*four*five*six*seven*eight*nine ==0:
        #print('there are not enough values in one of the columns to be meaningful')
        return 0,0
    else: return expectedQuantities, observedQuantities

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def remove_NaN_values_from_list(list1):
    for i in list1:
        if math.isnan(i) == True:
            list1.remove(i)
    
def remove_nonnumeric_values_from_list(list1):
    for i in list1:
        if isinstance(i, datetime.date) == True:
            list1.remove(i)
    for i in list1:
        if is_number(i) == False:
            list1.remove(i)
    for i in list1:
        if i == 0:
            list1.remove(i)
        
##this is my parent method##
def get_ratio_each_variable(filePath):
    df1 = pd.read_excel(filePath)
    df2 = pd.DataFrame()
    df2['1st_Sig_Fig'] = nl.values
    df2.set_index('1st_Sig_Fig', inplace=True)
    columnNameList = list(df1.columns.values.tolist())
    i = 0
    for column in df1:
        list1 = df1[column].tolist()
        remove_nonnumeric_values_from_list(list1)
        remove_NaN_values_from_list(list1)
        list2 = compile_ratio_of_first_digits(list1)
        se = pd.Series(list2)
        columnName = columnNameList[i]
        df2[columnName] = se.values
        i = i+1
    return df2

def get_quantity_each_variable(filePath):
    df1 = pd.read_excel(filePath)
    eDf = pd.DataFrame()
    eDf['1st_Sig_Fig'] = nl.values
    eDf.set_index('1st_Sig_Fig', inplace=True)
    oDf = pd.DataFrame()
    oDf['1st_Sig_Fig'] = nl.values
    oDf.set_index('1st_Sig_Fig', inplace=True)
    columnNameList = list(df1.columns.values.tolist())
    i = 0
    for column in df1:
        list1 = df1[column].tolist()
        remove_nonnumeric_values_from_list(list1)
        remove_NaN_values_from_list(list1)
        expectedList, observedList = compile_quantity_of_first_digits(list1)
        if expectedList ==0 and observedList == 0:
            print("REMARK: there are not enough data in "+columnNameList[i]+" to be meaningful")
        else:
            eSe = pd.Series(expectedList)
            oSe = pd.Series(observedList)
            columnName = columnNameList[i]
            eDf[columnName] = eSe.values
            oDf[columnName] = oSe.values
        i = i+1
    return eDf, oDf

def chi_square_test(eDf, oDf):
    columnNameList = list(eDf.columns.values.tolist())
    x2List = list()
    violationList = list()
    for column in eDf:
        eList = eDf[column].tolist()
        oList = oDf[column].tolist()
        x2 = 0
#        for i in range(len(eList)):
#            number1 = eList[i]
#            e = float(number1)
#            number2 = oList[i]
#            o = float(number2)
        i = 0
        for item in eList:
            e = eList[i]
            o = oList[i]
            xPart = (((o-e)*(o-e)) / e)
            x2 = x2+xPart
            i = i+1
        x2List.append(x2)
        if x2 >= 15.507:
            violationList.append(True)
        elif x2 < 15.507:
            violationList.append(False)
        else: print('something went wrong w/ chi-square list compiation')
    violationTable = pd.DataFrame()
    cnl = pd.Series(columnNameList)
    x2l = pd.Series(x2List)
    vl = pd.Series(violationList)
    violationTable['Column Names'] = cnl.values
    violationTable.set_index('Column Names', inplace = True)
    violationTable['X2 Value'] = x2l.values
    violationTable['Violation?'] = vl.values
    return violationTable
    
    
    


#excelPath = "\\\\Na.ad.guycarp.com\\us_users\\SEA\\U1189580\\foofi.xlsx"
#excelPath = "\\\\Na.ad.guycarp.com\\us_users\\SEA\\U1189580\\ms wc.xlsx"
#df3 = get_ratio_each_variable(excelPath)
#print(df3)
#df3.plot(kind = 'bar', ax=ax,figsize = (10,10))
#####################window loop########
window = Tk()
window.title("Fact Checking with Benford's Law")
l1 = Label(window, text="This program seeks to fact-check datasets",font=("Arial Bold",12))
l1.pack()
l2 = Label(window, text="by leveraging Benford's Law. According to",font=("Arial Bold",12))
l2.pack()
l3 = Label(window, text="Benford's Law, the distribution of the first",font=("Arial Bold",12))
l3.pack()
l4 = Label(window, text="significant digit of the numbers in most",font=("Arial Bold",12))
l4.pack()
l5 = Label(window, text="datasets follows an inverse logarithmic scale.",font=("Arial Bold",12))
l5.pack()

l6 = Label(window, text="Select an Excel (.xlx) file to check.",font=("Arial Bold",12))
l6.pack()
l7 = Label(window, text="This version defaults to the first tab.",font=("Arial Bold",12))
l7.pack()

##TODO - add a spinbox that specifies which tab to check
#labelRow = Spinbox(window, from_=0,to=100, width=5)
#labelRow.pack()

appendBtn = Button(window, text='Select Workbook to Check', command=lambda:clicked())
quitBtn = Button(window, text='Exit Program', command=lambda:window.quit())
appendBtn.pack()
quitBtn.pack()

t1 = Text(window)
t1.pack()

class PrintToT1(object):
    def write(self, s):
        t1.insert(END, s)
sys.stdout = PrintToT1()

def clicked():
    try:
        window.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("Excel files","*.xlsx"),("all files","*.*")))
        filePath1 = window.filename
        #filePath1 = filePath1.Replace("\\","/")
    except Exception as e: messagebox.showinfo("Status","Problem with the file; make sure it's .xlsx \n "+e)
    try:
        df3 = get_ratio_each_variable(filePath1)
        #messagebox.showinfo("Outcome",df3)
        print("The following table outlines the ratio of first sig figs")
        print(df3)
    except Exception as e: messagebox.showinfo("Status",e)
    print("next we'll apply a chi-square test between our expected Benford values")
    print("(i.e. .301,.176,.125,.097,.079,.067,.058,.051,.046 respectively)")
    print("against our outcome. This will test the hypothesis that")
    print("one of our variables violates Benford's Law.")
    print("The first table below displays expected values, the second displays observed")
    try:
        eDf, oDf = get_quantity_each_variable(filePath1)
        print(eDf)
        print(oDf)
        chiSquareTable = chi_square_test(eDf, oDf)
        print("The following table displays violations of the Chi Square Test")
        print(chiSquareTable)
    except Exception as e: messagebox.showinfo("Status", "Problem with quantity compilation "+e)
    try:
        graphDis = df3.plot(kind = 'bar', ax=ax,figsize = (10,10))
        saveLocation = filedialog.asksaveasfilename(filetypes=(("PNG", "*.PNG"),
                                                         ("All files", "*.*") ))
#        graphDis.savefig(saveLocation)
        fig = graphDis.get_figure()
        fig.savefig(saveLocation)
        messagebox.showinfo("Status", "A distribution graph is saved here \n"+saveLocation)
        #messagebox.showinfo("Outcome",graphDis)
    except Exception as e: messagebox.showinfo("Status",e)
    
    
window.mainloop()

#sys.exit(0)

import successful
